In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

1) Contamos con un cluster que tiene 4 computadoras. Queremos aprovechar el paralelismo del cluster para calcular los números primos entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo (2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13...) luego eliminamos todos los mayores a 3 divisibles por 3 y nos queda (2,3,5,7,11,13....etc) luego todos los divisibles por 5 y así sucesivamente. El resultado final es una lista de números que son primos. Programar este programa usando PySpark (**) (15

In [345]:
l= list(range(2, 20))
rdd = sc.parallelize(l,4)
pivote = 2
nuevoPivote = 2
continuar = True


while(continuar):
    try : 
        nuevoPivote = rdd.filter(lambda x: x > pivote).reduce(lambda x,y : x if x<y else y)
        #Esto lo meto aca porque sino en algun momento va a querer hacer reduce de una lista vacia
        continuar = True
    except:
        continuar = False
    print rdd.collect()
    print pivote
    rdd = sc.parallelize(rdd.filter(lambda x:((x<=pivote)or(0 != (x%pivote)))).collect())
    #hago esta mierda porque rdd = rdd.filter hacia lo que queria
    print rdd.collect()
    pivote = nuevoPivote

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
2
[2, 3, 5, 7, 9, 11, 13, 15, 17, 19]
[2, 3, 5, 7, 9, 11, 13, 15, 17, 19]
3
[2, 3, 5, 7, 11, 13, 17, 19]
[2, 3, 5, 7, 11, 13, 17, 19]
5
[2, 3, 5, 7, 11, 13, 17, 19]
[2, 3, 5, 7, 11, 13, 17, 19]
7
[2, 3, 5, 7, 11, 13, 17, 19]
[2, 3, 5, 7, 11, 13, 17, 19]
11
[2, 3, 5, 7, 11, 13, 17, 19]
[2, 3, 5, 7, 11, 13, 17, 19]
13
[2, 3, 5, 7, 11, 13, 17, 19]
[2, 3, 5, 7, 11, 13, 17, 19]
17
[2, 3, 5, 7, 11, 13, 17, 19]
[2, 3, 5, 7, 11, 13, 17, 19]
19
[2, 3, 5, 7, 11, 13, 17, 19]


 Se tiene un archivo que representa una red social con
billones de links entre usuarios de la forma (id1,id2). Se
quieren encontrar todos los triángulos de tipo (id1,id2,id3)
tales que existen (id1,id2),(id2,id3),(id1,id3). Explicar, sin
programar, que algoritmo usaría para listar todos los triángulos
que existen en la red social usando Map Reduce. En concreto
se pide para el siguiente fragmento del archivo:
(A,B) (B,C) (A,C) (A,D) (C,D) (D,E) (D,F) (E,F) (A,E)
- Qué recibe el metodo Map, qué hace y qué emite.
- Qué recibe el metodo Reduce, qué hace y qué emite.
Si lo hace en dos o mas iteraciones entonces explique los
puntos anteriores por cada iteración. (****) (15pts)
(opcional x puntos extras) ¿De qué forma podría usarse el
contar la cantidad de triángulos para saber si un determinado
grafo es o no una red social? Justifique. (*****) (5 pts)

In [ ]:
l = [(1,2),(1,3),(2,3),(2,4),(3,4),(3,5),(4,5),(5,6)]
print l

rdd = sc.parallelize(l)
rdd.collect()

In [ ]:
def f(l):
    comb = []
    for i in range(0, len(l[1])):
        for j in range(i+1, len(l[1])):
                comb += [sorted([l[0],l[1][i],l[1][j]])]
    return comb


print rdd.map(lambda x: [(x[0],x[1]),(x[1],x[0])]).flatMap(lambda x : x)\
.map(lambda x : (x[0],[x[1]])).reduceByKey(lambda x,y : x+y).map(lambda x: f(x)).flatMap(lambda x:x).map(lambda x: ((x[0],x[1],x[2]),1)).reduceByKey(lambda x,y : x+y).filter(lambda x: x[1] >2).collect()




In [280]:
l = sc.parallelize([1,2,3,4,23])
print l.reduce(lambda x,y:x if x>y else y)

23


Se tiene un RDD con las coordenadas de rectángulos de la forma 
(x1,x2,y1,y2). Se pide programar en PySpark un programa que
encuentre el rectángulo de superficie mínima que contiene al punto(w,z) (*?) (15 pts

In [285]:
rect = sc.parallelize([(2,5,4,8),(3,10,3,7),(1,7,1,6),(4.5,6,3.4,5),(2.5,4,3.2,4.1),(2,3,2,3.1)])
print rect.collect()
p = (3,3.4)
print p

def f(x,p):
    return ((p[0]>=x[0] and p[0]<=x[1] and p[1] >= x[2] and p[1] <= x[3]))

def area(x):
    return (abs(x[0]-x[1]) * abs(x[2]-x[3]))

print rect.filter(lambda x: f(x,p)).map(lambda x: (x,area(x))).reduce(lambda x,y: x if x[1] < y[1] else y)
# .collect()




[(2, 5, 4, 8), (3, 10, 3, 7), (1, 7, 1, 6), (4.5, 6, 3.4, 5), (2.5, 4, 3.2, 4.1), (2, 3, 2, 3.1)]
(3, 3.4)
((2.5, 4, 3.2, 4.1), 1.3499999999999992)


1) Dada una colección de documentos queremos encontrar
frases de 1 , 2 o 3 palabras que sean anagramas de otras. Por
ejemplo: (“Postmaster”, “Stamp store”) o (“A telescope” , “To
see Place”) o (“The cockroach”, “cook catch her”). Esta tarea
implica una combinatoria muy difícil por lo que se decide usar
Map-Reduce para paralelizarla. Usando Map-Reduce programar
la solución a este problema listando todos los pares de
anagramas entre frases de 1, 2 o 3 palabras. Como puede
verse en los ejemplos se ignoran mayúsculas y minúsculas y
los espacios en blanco, puntuación, etc. Suponer que existe la
función word_tokenizer que recibe un texto y devuelve un
vector de palabras ya convertidas a minúsculas y sin
puntuación

In [404]:
rdd = sc.parallelize(['postmaster see cook',
                      'stamp store a telescope',
                      'to see place cook catch her',
                      'the cockroach die with you'])

def frases(x):
    w = []
    x = x.split()
    for i in range(0, len(x)):
        w += [[x[i]]]
    size = len(x)
    for i in range(0,size-1):
        w += [[x[i]+' '+ x[i+1]]]
    for i in range(0, size-2):
        w += [[x[i]+' '+x[i+1]+' '+x[i+2]]]
    
    return w

def sortWord(s):
    s = sorted(s[0])
    l = []
    for i in range(0,len(s)):
        if s[i] != ' ':
            l += s[i]
    s = ''
    for i in range(0,len(l)):
        s += l[i]
    return s
    
print rdd.map(lambda x: frases(x)).flatMap(lambda x: x).map(lambda x: (sortWord(x),[x[0]])).reduceByKey(lambda x,y :x+y).filter(lambda x :(len(x[1]) > 1) and (len(set(x[1])) >=len(x[1]))).collect()
# .collect()
# .map(lambda x: (x[0],).collect()

['postmaster see cook', 'stamp store a telescope', 'to see place cook catch her', 'the cockroach die with you']
[('aemoprsstt', ['postmaster', 'stamp store']), ('acccehhkoort', ['cook catch her', 'the cockroach']), ('aceeelopst', ['a telescope', 'to see place'])]


1) Se tiene un archivo con información sobre visitas a páginas
web de la forma: (URL, visitas, fecha). Existe solo un registro
por día para cada URL. Se quiere generar un archivo que, por
cada URL, indique cuál fue la fecha en la que tuvo mas visitas
y la cantidad de visitas.
Programar lo pedido en Map Reduce usando agregación para
minimizar la cantidad de datos que deben transferirse en la
red.
Atención: La resolución es muy simple, trivial, asi que es
fundamental resolver la agregación para el puntaje completo.
(**) (15 pts)

In [93]:
# (URL,vistas, fecha)

rdd = sc.parallelize([
    ('google',213,'23-2'),
    ('google',2232313,'23-2'),
    ('google',213,'22-2'),
    ('google',213,'21-2')
    ])
print rdd.map(lambda x: ((x[0],x[2]),x[1])).reduceByKey(lambda x,y : x+y).reduce(lambda x,y: x if x[1]>y[1] else y)


(('google', '23-2'), 2232526)


In [98]:
# Solucion usando aggregate

print rdd.map(lambda x: ((x[0],x[2]),x[1])).aggregateByKey(0, lambda x,y: x + y , lambda x,y : x+y ).takeOrdered(3,lambda x: -x[1])
# reduce(lambda x,y: x if x>y else y)


[(('google', '23-2'), 2232526), (('google', '22-2'), 213), (('google', '21-2'), 213)]


Se tiene un grafo dirigido representado mediante una lista
de adyacencias. Dado un nodo inicial y un nodo final queremos
que programe usando Map-Reduce un algoritmo que indique
cuál es el camino mínimo entre ambos nodos. En clase vimos
como calcular la longitud, ahora queremos saber cuál es el
camino.

In [175]:
rdd = sc.parallelize([(1,2),(1,3),(1,4),(2,4),(2,5),(3,5),(4,3),(4,5)])
# rdd.collect()

rdd = rdd.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x+y)
# rdd.collect()

def f(r,x,y):
    path = []
    adjsX = r.filter(lambda a: a[0] == x ).collect()
    adjsX = adjsX[0]
    print adjsX
    
    adjsDeLosAdjsDeX = []
    print r.collect()
    for i in range(0, len(adjsX[1])):
        path.append([x,adjsX[1][i]])
        
        l = r.filter(lambda a: a[0] == adjsX[1][i]).collect()
        if (l):
            adjsDeLosAdjsDeX.append(l[0][1])
    print adjsDeLosAdjsDeX
    print path
    

    for i in range(0, len(adjsDeLosAdjsDeX)):
        path[i] += adjsDeLosAdjsDeX
        

f(rdd,2,5)

# TERMINAR

(2, [4, 5])
[(4, [3, 5]), (1, [2, 3, 4]), (2, [4, 5]), (3, [5])]
[[3, 5]]
[[2, 4], [2, 5]]


En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras).
Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable.
Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2).
Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento.
Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0)

In [4]:
rdd1 = sc.parallelize([(1,'Hoy es lunes 12'),
        (2, 'tenemos entrega de taller'),
        (3,'la entrega esta como el orto'),
        (4,'lunes parcial datos')])



In [337]:
rdd2 = sc.parallelize([('id1',"taller",4),
        ('id1','Hoy',12),
        ('id1', 'lunes',-19),
        ('id2', 'lunes', -30),
        ('id3','datos',20),
        ('id2','datos',8),
        ('id3','lunes',-20)])